In [15]:
# Module Loading
from Korpora import Korpora                             # Open Korean DataSet
from konlpy.tag import *                                # 형태소 분석기
import spacy                                            # 형태소 분석기
from torch.utils.data import Dataset, DataLoader

In [4]:
import pandas as pd
import numpy as np


In [19]:
from konlpy.tag import Okt

In [52]:
import re
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
## 데이터 로딩
nsmc = Korpora.load("nsmc")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\KDP-27\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\KD

In [11]:
nsmc.train[0], nsmc.test[0]

(LabeledSentence(text='아 더빙.. 진짜 짜증나네요 목소리', label=0),
 LabeledSentence(text='굳 ㅋ', label=1))

In [12]:
# Test 데이터셋을 DataFrame으로 로딩
nsmcDF=pd.DataFrame(nsmc.test)          # 데이터양이 많아서 test로 확인, 원래는 train데이터 변환해야함
nsmcDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [45]:
n=0
for feature,label in nsmcDS:
    print(feature,label)
    n+=1
    if n==10:
        break

굳 ㅋ 1
GDNTOPCLASSINTHECLUB 0
뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아 0
지루하지는 않은데 완전 막장임... 돈주고 보기에는.... 0
3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠?? 0
음악이 주가 된, 최고의 음악영화 1
진정한 쓰레기 0
마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다 0
갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다 0
이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네.. 1


In [46]:
okt=Okt()

In [47]:
### 텍스트 Dataset에서 처리할 데이터셋만 로딩해서 토큰화 진행하는 반복자 함수
def generateToken(dataset):
    tokenlists=[]
    for text,label in dataset:
        # 한글빼고 다지우기
        text=re.sub('[^ㄱ-ㅎ가-힣]+',' ',text)
        tokenlist=''
        # 기본형으로 토큰 주세요
        tokens=okt.pos(text,norm=True,stem=True)
        for token in tokens:
            if (len(token[0])>1):
                tokenlist=tokenlist+' '+str(token[0])
        tokenlists.append(tokenlist)
    return tokenlists


In [49]:
## Test 데이터셋을 DataFrame으로 로딩
nsmcDF=pd.DataFrame(nsmc.test)
nsmcDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [ ]:
class TextDataset(Dataset):
    def __init__(self,feature,label):
        self.feature=feature
        self.label=label
        self.length=feature.shape[0]

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        return self.feature.iloc[index],self.label.iloc[index]

In [50]:
# NSMC의 데이터셋 인스턴스 생성
nsmcDS=TextDataset(nsmcDF['text'],nsmcDF['label'])

In [51]:
# 수치화 패딩
def vectorize(tokenlist,padding=20):
    ### 단어 사전 생성 및 변환
    cv=CountVectorizer()
    cv.fit_transform(tokenlist)
    vo=cv.vocabulary_
    vectorlist=tokenlist.copy()
    for idx,sen in enumerate(vectorlist):
        senlist=sen.split(' ')[1:]
        length=len(senlist)
        if length<padding:
            for ind,st in enumerate(senlist):
                senlist[ind]=vo[st]
            vectorlist[idx]=senlist+([0]*(padding-length))
        else:
            for ind,st in enumerate(senlist):
                senlist[ind]=vo[st]
            vectorlist[idx]=senlist[:padding]

    return vectorlist

In [57]:
tokenlist=generateToken(nsmcDS)


In [61]:
tokenlist

[' 굳다',
 '',
 ' 평점 나쁘다 않다 짜다 더욱 아니다',
 ' 지루하다 않다 완전 막장 주다 보기 에는',
 ' 아니다 다섯 주다 나오다 심기 불편하다 하다',
 ' 음악 주가 되다 최고 음악 영화',
 ' 진정하다 쓰레기',
 ' 마치 미국 애니 에서 튀어나오다 창의력 없다 로봇 디자인 부터가 고개 젖다 하다',
 ' 갈수록 개판 되다 중국영화 유치하다 내용 없다 잡다 끝나다 안되다 무기 유치하다 남무 그리다 동사서독 같다 영화 류작 이다',
 ' 이별 아픔 찾아오다 새롭다 인연 기쁨 모든 사람 그렇다 않다',
 ' 괜찮다 오랜 포켓몬스터 잼밌',
 ' 한국 독립영화 한계 그렇게 아버지 되다 비교 되다',
 ' 청춘 아름답다 아름답다 이성 흔들다 놓다 찰나 아름답다 자다 포착 섬세하다 아름답다 수채화 같다 퀴어 영화 이다',
 ' 보이다 반전 이다 영화 흡인 사라지다 않다',
 ' 스토리 연출 연기 비주 영화 기본 조차 안되다 영화 무슨 이렇다 영화 찍다 김문옥 감독 영화 경력 인데 조무래기 영화 평론 하다 같다 마인드 빠지다',
 ' 소위 가라 평점',
 ' 최고',
 ' 발연기 도저히 보다 진짜 이렇게 연기 하다 상상 하다',
 ' 나이스',
 ' 재미 없다 우려 먹다 챔프 에서 방송 몇번 하다 ㅋㅋㅋ',
 ' 금요일 나이트메어 시리즈 함께 가장 많다 시리즈 양산 해내다 헬레 시리즈 작가 상상력 돋보이다 작품 이며 갈고리 사지 찢다 고어씬 지금 보더 라도 상당하다 잔인하다 충격 이다',
 ' 나름 교훈 돋다 하지만 어쩔 수없이 평점 받다 없다 섹스 코미디',
 ' 재밌다 보다 영화 이다',
 ' 민주화 시대 억눌리다 영혼 관음 욕구 분출 인상 이다',
 ' 일본 천황 미국 으로부터 받다 면죄부 긴박하다 과정 루즈 하고 지저분하다 늘어놓다',
 ' 괜히 나오다 어미 건드리다 긁다 부스 만들다 아래 보이다 영화 제대로 보다 끄다 새끼 나오다 하다 그냥 가면 한테 당하다 것임 하다 나오다 뻔하다 작살 밖에 없다',
 ' 번은 보다 어쩜 이렇게 만들다

In [58]:
data_length = [len(data) for data in tokenlist]

data_length
MAX_LENGTH = max(data_length)

MAX_LENGTH

162

In [63]:
vectorlist=vectorize(tokenlist,padding=20)

In [64]:
vectorlist

[[2066, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [22607,
  3282,
  13527,
  19944,
  4741,
  13097,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [19602,
  13527,
  15375,
  6714,
  19221,
  9257,
  14356,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [13097,
  4236,
  19221,
  3305,
  12785,
  9882,
  23117,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [16520,
  19201,
  5237,
  20673,
  16520,
  14923,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [19811, 13003, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [6656,
  7944,
  13683,
  14399,
  22076,
  20274,
  14298,
  6180,
  5525,
  9688,
  1286,
  18752,
  23117,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [455,
  807,
  5237,
  19422,
  16255,
  3632,
  14298,
  17883,
  3172,
  13434,
  7625,
  16255,
  3515,
  2281,
  5173,
  657,
  14923,
  6333,
  167

In [65]:
for i in range(10):
    print(tokenlist[i])
    print(vectorlist[i])

 굳다
[2066, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 평점 나쁘다 않다 짜다 더욱 아니다
[22607, 3282, 13527, 19944, 4741, 13097, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 지루하다 않다 완전 막장 주다 보기 에는
[19602, 13527, 15375, 6714, 19221, 9257, 14356, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 아니다 다섯 주다 나오다 심기 불편하다 하다
[13097, 4236, 19221, 3305, 12785, 9882, 23117, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 음악 주가 되다 최고 음악 영화
[16520, 19201, 5237, 20673, 16520, 14923, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 진정하다 쓰레기
[19811, 13003, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 마치 미국 애니 에서 튀어나오다 창의력 없다 로봇 디자인 부터가 고개 젖다 하다
[6656, 7944, 13683, 14399, 22076, 20274, 14298, 6180, 5525, 9688, 1286, 18752, 23117, 0, 0, 0, 0, 0, 0, 0]
 갈수록 개판 되다 중국영화 유치하다 내용 없다 잡다 끝나다 안되다 무기 유치하다 남무 그리다 동사서독 같다 영화 류작 이다
[455, 807, 5237, 19422, 16255, 3632, 14298, 17883, 3172, 13434, 7625, 16255, 3515, 2281, 5173, 657, 14923, 6333, 16703, 0]
 이별 아픔 찾아오다 새롭다